<a href="https://colab.research.google.com/github/hacksaremeta/IS-Sentence-Completion/blob/model/is_autocomplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence Completion (TUD IS Project)

<a id="toc"></a>
## Table of contents
* 1 [Introduction (TODO)](#introduction)  
* 2 [Training data](#data_preparation)  
    * 2.1 [DataManager class](#data_manager)  
    * 2.2 [DataUtils class](#data_utils)  
* 3 [Neural Network Implementation](#impl_nn)  
    * 3.1 [Data preparation](#keras_preparation)  
    * 3.2 [Creating a neural network](#keras_create_rnn)
    * 3.3 [Training the neural network](#keras_train_rnn)
    * 3.4 [Evaluating the neural network (this will be removed in future)](#keras_evaluate_rnn)
    * 3.5 [Making Predictions](#keras_predict_rnn) 
* 4 [Evaluation](#evaluation)
    * 4.1 [Dataset comparison](#eval_dataset)
    * 4.2 [Data preparation comparison](#eval_preparation)
    * 4.3 [Network comparison](#eval_rnn)
    * 4.4 [Conclusion](#eval_conclusion)

<a id="data_preparation"></a>
## Training data

In order to fetch data from PubMed and save it into different datasets as well as to load those datasets, some functionality is needed. This functionality will be provided by the [DataManager class](#data_manager).
The loaded dataset then has to be prepared for training the neural network. This includes tokenization, label and feature extraction and encoding, all of which is handled by the [DataUtils class](#data_utils).
TODO: more explanation / documentation ...

<a id="data_manager"></a>
### DataManager Class

- Provides functionality regarding data including fetch, persistence and TF2/Keras preparation utils

In [ ]:
!pip install biopython

In [1]:
import os, json, logging, string
from Bio import Entrez, Medline

In [2]:
class DataManager():
    """Provides fetch, save and load functionality for datasets in json format"""
    
    def __init__(self, email, root_dir):
        self.email = email
        self.root_dir = root_dir
        self.log = logging.getLogger(self.__class__.__name__)

    def _exists_dataset(self, name):
        """Checks whether a dataset with the given name exists"""
        if not os.path.isdir(self.root_dir):
            return False
            
        for file in os.listdir(self.root_dir):
            if file.endswith(".json"):
                with open(os.path.join(self.root_dir, file), 'r') as f:
                    content = json.load(f)
                    if content["name"] == name:
                        return True
        return False

    def _fetch_papers(self, query : str, limit : int) -> 'list[dict]':
        """Retrieves data from PubMed"""
        Entrez.email = self.email
        record = Entrez.read(Entrez.esearch(db="pubmed", term=query, retmax=limit))
        idlist = record["IdList"]
        self.log.info("Found %d records for %s." % (len(idlist), query.strip()))
        records = Medline.parse(Entrez.efetch(db="pubmed", id=idlist, rettype="medline", retmode = "text"))
        return [r for r in records if "AB" in r]

        
    def create_dataset(self, query : str, name : str, limit=50, overwrite=False) -> None:
        """
        Wraps other methods in this class
        Creates a dataset from multiple queries
        Does nothing if the dataset is already present (param overwrite)
        Limits every query to <limit> results
        """
        exists_dataset = self._exists_dataset(name)
        if not exists_dataset or (exists_dataset and overwrite):
            self.log.info("Dataset does not exist, fetching from PubMed...")
            q_data = dict()
            q_data["query"] = query
            papers = self._fetch_papers(query, limit)
            list_of_abstracts = [p["AB"] for p in papers]
            q_data["abstracts"] = list_of_abstracts
            res = {"name": name}
            res["data"] = list()
            res["data"].append(q_data)
            self._save_dataset(res, name)
        else:
            self.log.info("Dataset already exists, skipping fetch")

    def _save_dataset(self, dataset: dict, name : str) -> None:
        """
        Creates a file <name>.json in the dataset directory
        For JSON file structure see below
        Param dataset has a structure analogous to the JSON file
        """
        if not os.path.isdir(self.root_dir):
            os.makedirs(self.root_dir)

        with open(os.path.join(self.root_dir, name + ".json"), 'w') as f:
            json.dump(dataset, f, indent=2)
        
    def load_full_dataset(self, name : str) -> 'list[str]':
        """
        Finds the file that matches given <name> in JSON information,
        parses it, loading all abstracts into a list (one string for each abstract)
        and returns it (Error if dataset doesn't exist)
        """

        if  not self._exists_dataset(name):
            self.log.info("Dataset does not exist")
            
        else:
           with open(os.path.join(self.root_dir, name+'.json'), 'r') as file:
                abstract_list=[]
                jsonObject = json.load(file)
                data_list= jsonObject['data']
                for item in data_list:
                    abstract_list.extend(item['abstracts'])
                return abstract_list

    def load_query_from_dataset(self, name : str, query : str) -> 'list[str]':
        """Like load_full_dataset but only loads abstracts for a single query"""


        result = self._exists_dataset(name)

        if  result:

            with open(os.path.join(self.root_dir, name+'.json'), 'r') as file:

                query_abstracts=[]
                jsonObject = json.load(file)
                data_list= jsonObject['data']

                q_names = [x['query'] for x in data_list]

                if query not in q_names:
                    self.log.info("The Query that you are searching for,does not exist in the Dataset")
                else:

                      for queries in data_list:
                            if queries['query'] == query:
                              query_abstracts.extend(queries['abstracts'])
                              return query_abstracts

        else:
             self.log.info("Dataset does not exist")


    def remove_punctuation(self, name:str) -> 'list[str]':


            abstracts_list= self.load_full_dataset(name)

            for text in abstracts:

                text = text.translate(str.maketrans('', '', string.punctuation))
                abstracts_list.append(text)


            return  abstracts_list

<a id="data_utils"></a>
### DataUtils Class

[Back to TOC](#toc)
- Static class providing utility functions to prepare data for training

In [3]:
import numpy as np
import random
from typing import Any
from sklearn.model_selection import train_test_split

In [4]:
# TODO: unify method param types (all np.array instead of list)
class DataUtils():
    """Provides utility functions for data preparation"""
    
    @staticmethod
    def extract_features_and_labels(sequences : 'list[list[Any]]',
                                            train_interval : 'tuple[int, int]') -> 'tuple[list[Any], list[Any]]':
        """
        Choses a random number l from <train_interval> (chosen for every sequence) and extracts
        features of dynamic length l from every sequence
        Every l+1-th word is extracted as a label
        Reurns tuple(features, labels)
        """
        features = []
        labels = []
        for s in sequences:
            l = random.randrange(*train_interval, 1)
            for i in range(l, len(s)):
                # First l words are features
                features.append(s[i-l : i])
                
                # l-th word is label
                labels.append(s[i])
        
        return (features, labels)
        
    @staticmethod
    def encode_data(labels : 'list[Any]', num_code_words : int) -> np.array:
        """
        One-hot encode labels using numpy to
        improve the training speed of the network
        """

        # Use numpy for better compatibility and performance
        # Data type: 8bit integers for binary numbers (0, 1)
        # Could be optimized in space by using single bits instead
        # But that adds overhead in calculation (tradeoff time - space)
        # Since we want improved training speed we just use
        # numpys smallest data type byte/uint8 here
        labels_encoded = np.zeros((len(labels), num_code_words), dtype=np.uint8)

        # One-hot encode
        for i, word in enumerate(labels):
            labels_encoded[i, word] = 1
            
        return labels_encoded
    
    # Uses Scikit-learn here for simplicity
    @staticmethod
    def split_data(features: np.array, labels: np.array, _test_size=0.2) -> Any:
        """
        Splits features and labels into training and validation data sets
        Returns: (features_training, features_validation, labels_training, labels_validation)
        """
        return train_test_split(features, labels, test_size=_test_size)

<a id="impl_nn"></a>
## Neural Network Implementation (LSTM RNN)  
For the general methodology regarding Keras neural networks see [Tensorflow Docs: Text generation with an RNN](https://www.tensorflow.org/text/tutorials/text_generation), [Sanchit Tanwar: Building our first neural network in keras](https://towardsdatascience.com/building-our-first-neural-network-in-keras-bdc8abbc17f5) and [Will Koehrsen: Recurrent Neural Networks by Example in Python](https://towardsdatascience.com/recurrent-neural-networks-by-example-in-python-ffd204f99470).
In this case the sequences given are words instead of characters and the RNN predicts the next word.
Therefore we use the Keras Tokenizer to convert sentences to vectors of word representatives (integers).
After tokenization each 'word' will be converted to a feature vector using Keras pre-trained embeddings.
Then we train the network by giving it n 'words' (features) from the PubMed training data and having it predict the (n+1)-th word (label) in the sequence.
The predicted word is then compared to the actual word present in the training data and back-propagation is used to tweak the network layers.

<a id="keras_preparation"></a>
### Data preparation

[Back to TOC](#toc)

In [5]:
import re, pickle, warnings
from tensorflow import device
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.compat.v1.logging import set_verbosity, ERROR
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Masking, Dense, Dropout, Flatten, Input

In [6]:
# Suppress TF2 GPU Warnings
set_verbosity(ERROR)

In [7]:
if __name__ == "__main__":
    # Init logging
    logging.basicConfig(level=logging.DEBUG, format='[%(levelname)s] %(name)s: %(message)s')
    log = logging.getLogger("Main")

    # Create DataManager in 'res/datasets' folder
    data_folder = os.path.join("res", "datasets")
    dman = DataManager("mymail@example.com", data_folder)


    query = "clustering[ti] algorithm"
    records = 1000
    dataset_name = f"{query} Dataset {records}"
    #query = ["RNA", "mRNA", "tRNA"]
    #dataset_name = f"RNA Dataset"

    # Gather maximum of 100 abstracts for each query
    # I would suggest around 5 - 20 abstracts in total for the small data sets
    # and maybe 500 - 5000 for the final ones but we'll have to test
    # since that depends on how long it takes to train the network
    # This only queries PubMed if data if the data is not already present
    dman.create_dataset(query, dataset_name, records, overwrite=False)

    # Load the dataset
    abstracts = dman.load_full_dataset(dataset_name)
    abstracts_mrna = dman.load_query_from_dataset(dataset_name, query)

    ab = dman.remove_punctuation(dataset_name)

    assert(len(ab) > 0)
    log.debug(f"First extracted abstract: {ab[0]}")

    # Perform some regex preprocessing to improve data quality
    # Do this sequentially for better code readability
    
    # Separate punctuation to keep in tokenization 
    ab = [re.sub(r"([.:,;!?])", r" \1 ", w) for w in ab]
    
    # Remove URLs and Emails
    # Credit: Matthew O'Riordan
    # https://blog.mattheworiordan.com/post/13174566389/url-regular-expression-for-links-with-or-without
    regex_url_email = (r'((([A-Za-z]{3,9}:(?:\/\/)?)(?:[-;:&=\+\$,\w]+@)?'
                       r'[A-Za-z0-9.-]+|(?:www.|[-;:&=\+\$,\w]+@)[A-Za-z0-9.-]+)((?:\/[\+~%'
                       r'\/.\w\-_]*)?\??(?:[-\+=&;%@.\w_]*)#?(?:[.\!\/\\w]*))?)')
    ab = [re.sub(regex_url_email, '', w) for w in ab]
    
    # Remove references like [...], (...)
    ab = [re.sub(r"(\(.*?\)|\[.*?\])", '', w) for w in ab]
        
    # Substitute special Symbols like &, /, ...
    ab = [re.sub(r"([^\s]*)&([^\s]*)", r"\1 and \2", w) for w in ab]
    ab = [re.sub(r"([^\s]*)\/([^\s]*)", r"\1 or \2", w) for w in ab]
    
    # Remove duplicate spaces
    ab = [re.sub(r"\s\s+", ' ', w) for w in ab]
    
    assert(len(ab) > 0)
    log.debug(f"First extracted abstract after RegEx preprocessing: {ab[0]}")
    
    # Tokenize abstracts
    # See https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
    # Filters slightly modified (comp. to docs) to keep punctuation
    # Lowercase has to be used for pre-trained embeddings (filters='#$%&()*+-,<=>@[\\]^_`{|}~\t\n')
    # '"%;[\\]^_`{|}~\t\n'
    tokenizer = Tokenizer(
        num_words=None, 
        filters='"#*+<>@=~_|´`\\[]{}\t\n',
        lower = False, split = ' '
    )

    tokenizer.fit_on_texts(ab)

    # Generates list of lists of integers
    # Can be reversed with the sequences_to_texts() function of the tokenizer
    sequences = tokenizer.texts_to_sequences(ab)

    assert(len(sequences) > 0)
    log.debug(f"First tokenized sequence: {sequences[0]}")

    # Prepare data for input to RNN
    # Extract features and labels
    # Number of words before prediction:
    # random number from feature_len_range
    # Use tuple(n, n+1) for fixed feature length n
    feature_len_range = (60, 61)
    features, labels = DataUtils.extract_features_and_labels(sequences, feature_len_range)

    assert(len(features) > 0 and len(labels) > 0)
    log.debug(f"First extracted feature: {tokenizer.sequences_to_texts(features)[0]} {features[0]}")
    log.debug(f"First extracted label: {tokenizer.index_word[labels[0]]} [{labels[0]}]")

    # One-hot encode data for improved training performance
    num_code_words = len(tokenizer.index_word) + 1
    labels_encoded = DataUtils.encode_data(labels, num_code_words)

    assert(len(labels_encoded) > 0)
    log.debug(f"First one-hot encoded label: [0 ... {labels_encoded[0][labels[0]]} (at index {labels[0]}) ... 0]")

    # Final log for prepared data
    log.info(f"Loaded {labels_encoded.shape[0]} sequences"
             f" with an encoded length of ~{labels_encoded.shape[1] // 8} bytes per sequence")
    
    # Convert features to numpy array
    # This is necessary for input to the RNN
    features = np.array(features)
    
    # Split dataset into training and validation sets
    features_training, features_validation, labels_training, labels_validation = \
    DataUtils.split_data(features, labels_encoded, 0.2)
    
    assert(len(features_training) > 0 and len(features_validation) > 0 and
           len(labels_training) > 0 and len(labels_validation) > 0)
    log.info(f"Size of training data: {features_training.shape[0]} sequences")
    log.info(f"Size of validation data: {features_validation.shape[0]} sequences")
    
    log.info("Training data preparation finished")

[INFO] DataManager: Dataset already exists, skipping fetch
[DEBUG] Main: First extracted abstract: In this research study, we first define the strong degree of a vertex in an m-polar fuzzy graph. Then we present various useful properties and prove some results concerning this new concept, in the case of complete m-polar fuzzy graphs. Further, we introduce the concept of m-polar fuzzy strength sequence of vertices, and we also investigate it in the particular instance of complete m-polar fuzzy graphs. We discuss connectivity parameters in m-polar fuzzy graphs with precise examples, and we investigate the m-polar fuzzy analogue of Whitney's theorem. Furthermore, we present a clustering method for vertices in an m-polar fuzzy graph based on the strength of connectedness between pairs of vertices. In order to formulate this method, we introduce terminologies such as A-reachable vertices in m-polar fuzzy graphs, A-connected m-polar fuzzy graphs, or A-connected m-polar fuzzy subgraphs (in ca

<a id="keras_create_rnn"></a>
### Creating the neural network

[Back to TOC](#toc)

In [8]:
# Network params
train_embedding = True
embed_vec_size = 200

num_nodes_lstm = 128
num_nodes_dense = 128

# Training params
num_batch = 2800
num_epochs = 150

X_train = features_training
y_train = labels_training
X_valid = features_validation
y_valid = labels_validation

# Model save params
model_name = "model-baseline"

# Create directories if necessary
model_dir = os.path.join("res", "models")
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

# Filename semantics:
# <model_name>_<train_size>_<output_dim>_<selftrained_embeddings>_<num_nodes_lstm>_<num_nodes_dense>_<num_epochs>.h5
model_file = model_name + "_" + str(X_train.shape[0]) \
    + "_" + str(embed_vec_size) + "_" + str(int(train_embedding)) \
    + "_" + str(num_nodes_lstm) + "_" + str(num_nodes_dense) \
    + "_" + str(num_epochs) + ".h5"

# To make predictions later the tokenizer has to be saved
tokenizer_dir = os.path.join("res", "tokenizers")
if not os.path.isdir(tokenizer_dir):
    os.makedirs(tokenizer_dir)
    
with open(os.path.join(tokenizer_dir, model_file[:-3] + ".pkl"), "wb") as f:
    pickle.dump(tokenizer, f)

def make_model(input_dim : int,
               embed_vec_size : int, nodes_lstm : int,
               nodes_dense : int, dropout_lstm : float = 0.1,
               dropout_lstm_recurrent : float = 0.1, dropout_dense : float = 0.5) -> Any:
    """
    Creates a sequential Keras Model with given parameters
    The embeddings have to be trained
    """
    model = Sequential()

    # Embedding layer
    # num_code_words = number of unique words
    # training_length we use the first num_pred words
    model.add(
        Embedding(input_dim = input_dim,
                  input_length = None,
                  output_dim = embed_vec_size,
                  trainable = True,
                  mask_zero = True))

    # Recurrent layer
    model.add(LSTM(nodes_lstm, return_sequences = False, 
                   dropout = dropout_lstm, recurrent_dropout = dropout_lstm_recurrent))

    # Fully connected layer
    model.add(Dense(nodes_dense, activation = 'relu'))

    # Dropout for regularization
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(input_dim, activation = 'softmax'))
    
    return model



def make_model_pretrained(input_dim : int,
               embed_vec_size : int, embedding_matrix : np.array,
               nodes_lstm : int, nodes_dense : int,
               dropout_lstm : float = 0.1, dropout_lstm_recurrent : float = 0.1,
               dropout_dense : float = 0.5) -> Any:
    """
    Creates a sequential Keras Model with given parameters
    Uses pre-trained embeddings
    """
    model = Sequential()

    # Allow for variable length input
    model.add(Input(shape = (None,)))
    
    # Embedding layer
    # num_code_words numb of unique words 
    # training_length we use the first num_pred words
    model.add(
        Embedding(input_dim = input_dim,
                  input_length = None,
                  output_dim = embed_vec_size,
                  weights = embedding_matrix,
                  trainable = False,
                  mask_zero = True))

    # Masking layer for pre-trained embeddings
    model.add(Masking(mask_value=0.0))

    # Recurrent layer
    model.add(LSTM(nodes_lstm, return_sequences = False, 
                   dropout = dropout_lstm, recurrent_dropout = dropout_lstm_recurrent))

    # Fully connected layer
    model.add(Dense(nodes_dense, activation = 'relu'))

    # Dropout for regularization
    model.add(Dropout(0.5))

    # Output layer
    model.add(Dense(input_dim, activation = 'softmax'))
    
    return model

    

# Create Keras model
if train_embedding:
    model = make_model(num_code_words,
                       embed_vec_size, num_nodes_lstm,
                       num_nodes_dense)
else:
    #model = make_model_pretrained(num_pred, num_code_words,
    #                              embed_vec_size, embedding_matrix,
    #                              num_nodes_lstm, num_nodes_dense)
    raise NotImplementedError

# Compile the model
model.compile(
    optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         3838000   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               168448    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 19190)             2475510   
Total params: 6,498,470
Trainable params: 6,498,470
Non-trainable params: 0
_________________________________________________________________


<a id="keras_train_rnn"></a>
### Training the neural network

[Back to TOC](#toc)

#### Callbacks

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

In [12]:
%load_ext tensorboard

In [11]:
train_log_dir = os.path.join("logs", "training", model_file[:-3])

def make_callbacks(model_name, save=True):
    """
    Creates callbacks for saving the model after each step
    and stopping once learning process is finished
    """
    callbacks = []
    
    stopping = EarlyStopping(monitor='val_loss', patience = 5)
    callbacks.append(stopping)
    
    reduce = ReduceLROnPlateau(monitor = 'loss', factor = 0.2,
                               patience = 5, min_lr = 0.0001)
    callbacks.append(reduce)
    
    board = TensorBoard(log_dir = train_log_dir, profile_batch = 0)
    callbacks.append(board)
    
    if save:
        checkpoint = ModelCheckpoint(
            os.path.join(model_dir, model_file),
            save_best_only = True,
            save_weights_only = False)
        callbacks.append(checkpoint)
        
    return callbacks

callbacks = make_callbacks(model_name)

In [ ]:
# On windows use 'tensorboard.exe --logdir logs/training' instead
%tensorboard --logdir "logs/training"

#### Training

In [12]:
# Train the model and save the 'best' version
history = model.fit(X_train,  y_train,  batch_size = num_batch, 
          epochs = num_epochs, callbacks = callbacks,
          validation_data = (X_valid, y_valid))

Train on 252276 samples, validate on 63070 samples
Epoch 1/150
252276/252276 [==============================] - 47s 185us/sample - loss: 7.6724 - acc: 0.0435 - val_loss: 7.0044 - val_acc: 0.0552
Epoch 2/150
252276/252276 [==============================] - 45s 178us/sample - loss: 6.9263 - acc: 0.0545 - val_loss: 6.9276 - val_acc: 0.0552
Epoch 3/150
252276/252276 [==============================] - 45s 178us/sample - loss: 6.8527 - acc: 0.0573 - val_loss: 6.8944 - val_acc: 0.0621
Epoch 4/150
252276/252276 [==============================] - 44s 176us/sample - loss: 6.7599 - acc: 0.0674 - val_loss: 6.7742 - val_acc: 0.0776
Epoch 5/150
252276/252276 [==============================] - 45s 177us/sample - loss: 6.6169 - acc: 0.0807 - val_loss: 6.6511 - val_acc: 0.0840
Epoch 6/150
252276/252276 [==============================] - 44s 176us/sample - loss: 6.4996 - acc: 0.0896 - val_loss: 6.8007 - val_acc: 0.0780
Epoch 7/150
252276/252276 [==============================] - 45s 177us/sample - loss:

<a id="keras_evaluate_rnn"></a>
### Evaluating the neural network (this will be removed in future)

[Back to TOC](#toc)

#### Loading the model

In [1]:
import os, time, pickle, warnings
from tensorflow import device
from tensorflow.keras.models import load_model
from tensorflow.compat.v1.logging import set_verbosity, ERROR

# Suppress TF2 GPU Warnings
set_verbosity(ERROR)

In [13]:
model_file = "model-baseline_156805_200_1_128_128_150.h5"

model_dir = os.path.join("res", "models")
tokenizer_dir = os.path.join("res", "tokenizers")

In [14]:
# Loading with TF2-DirectML on GPU fails for me
# so use CPU instead
with device('/cpu:0'):
    model = load_model(os.path.join(model_dir, model_file))

model.summary()
# model.evaluate(X_valid, y_valid, batch_size=4096, verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         2511400   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               168448    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 12557)             1619853   
Total params: 4,316,213
Trainable params: 4,316,213
Non-trainable params: 0
_________________________________________________________________


#### Visualization

In [ ]:
from matplotlib import pyplot as plt
logging.getLogger('matplotlib.font_manager').disabled = True

In [ ]:
accuracy_val = history.history['val_accuracy']
loss = history.history['val_loss']
epochs = range(0,num_epochs)
plt.plot(epochs, accuracy_val, 'g', label='Accuracy')
plt.plot(epochs, loss, 'b', label='Loss')
plt.title('Accuracy and Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

<a id="keras_predict_rnn"></a>
### Making Predictions

[Back to TOC](#toc)

In [15]:
from tensorflow.errors import UnimplementedError
import numpy as np
import warnings, logging

# Init logging
logging.basicConfig(level=logging.DEBUG, format='[%(levelname)s] %(name)s: %(message)s')
log = logging.getLogger("Main")

# Suppress np.log divide by 0 warnings
warnings.filterwarnings('ignore')

In [16]:
# Load tokenizer
with open(os.path.join(tokenizer_dir, model_file[:-3] + ".pkl"), "rb") as f:
    tokenizer = pickle.load(f)

In [22]:
# Randomness factor (in range [0, 1]
# 1.0 means no randomness
rand = 1.0

# Timeout for prediction in seconds
timeout = 2

# Fallback prediction length
# in case end of sentence is not predicted in time
pred_len = 10

# Get input sequence
s = [input("Enter a sentence to complete: ")]
sequences = np.array(tokenizer.texts_to_sequences(s))
len_orig = len(sequences)

# Make prediction
predictions = []
elem = ""
t_start = time.time()

# Stop on punctuation
end_symbols = ['.', '?', '!']
end_tokens = []
for t in end_symbols:
    try:
        t = tokenizer.word_index[t]
        end_tokens.append(t)
    except KeyError:
        continue

while elem not in end_tokens:
    try:
        pred = model.predict(sequences)[0]
    except UnimplementedError:
        log.warning("Input contains words that are not in vocabulary. Prediction will be inaccurate.")
    
    # Introduce random factor to prevent getting
    # stuck in prediction loop
    pred = np.exp(np.log(pred) / rand)
    
    # Issue: https://github.com/numpy/numpy/issues/8317
    # Solution: https://stackoverflow.com/a/53605818
    pred = np.asarray(pred).astype('float64')
    
    # Softmax
    pred = pred / pred.sum()
    
    # Pick one word from the
    # generated probability distribution
    probs = np.random.multinomial(1, pred, 1)[0]
    elem = np.argmax(probs)
    
    # Separately save predictions
    predictions.append(elem)
    
    # Append to current sequence for new input
    sequences = np.append(sequences, [[elem]], axis=1)
    
    # Timeout and fallback
    elapsed = time.time() - t_start
    if elapsed >= timeout:
        log.warning(f"Could not predict end of sentence within {timeout}s. Falling back to predicting {pred_len} words.")
        sequences = sequences[:len_orig + pred_len]
        predictions = predictions[:pred_len]
        break

# Convert model output to human-readable text
predictions = tokenizer.sequences_to_texts([predictions])[0]

log.info(f"Predicted sequence:\n{''.join(s[0])}\n{''.join(predictions)}")

Enter a sentence to complete: Clustering cells and depicting the lineage


[INFO] Main: Predicted sequence:
Clustering cells and depicting the lineage
The proposed algorithm is obtained with the corresponding clustering performance and population computational analysis is dramatically dedicated to other via supervised processing performance In addition the first sparsity based of VAE of parameters by the classification and quotient handwritten operation for misclassification or axial dataset They mixed-type the inherent filter results on the human Connectome Regularization , to a tree-based strategy .


<a id="evaluation"></a>
## Evaluation


<a id="eval_dataset"></a>
### Dataset comparison

[Back to TOC](#toc)

<a id="eval_preparation"></a>
### Data preparation comparison

[Back to TOC](#toc)

<a id="eval_rnn"></a>
### Network comparison

[Back to TOC](#toc)

<a id="eval_conclusion"></a>
### Conclusion

[Back to TOC](#toc)